<a href="https://colab.research.google.com/github/wurifah/Bisaai/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('NamedEntityRecognition.csv')
df.head(10)

,transaction_descriptor,store_number,dataset
0,DOLRTREE 2257 00022574 ROSWELL,2257,train
1,AUTOZONE #3547,3547,train
2,TGI FRIDAYS 1485 0000,1485,train
3,BUFFALO WILD WINGS 003,3,train
4,J. CREW #568 0,568,train
5,KRISPY KREME 0040 GREENVILLE SC,40,train
6,FIVE GUYS MN 1847 ECOM 612-339-9733 MN,1847,train
7,CASEYS GEN STORE 2650,2650,train
8,HUDDLE HOUSE 535,535,train
9,JAMBA JUICE #1305,1305,train


In [3]:
train = df[(df['dataset']=='train')|(df['dataset']=='validation')]
train

,transaction_descriptor,store_number,dataset
0,DOLRTREE 2257 00022574 ROSWELL,2257,train
1,AUTOZONE #3547,3547,train
2,TGI FRIDAYS 1485 0000,1485,train
3,BUFFALO WILD WINGS 003,3,train
4,J. CREW #568 0,568,train
...,...,...,...
195,ROSS STORES #15,15,validation
196,SPRINT STORE #346,346,validation
197,SPEEDWAY 07134 4343 OL,7134,validation
198,THE HOME DEPOT #8550,8550,validation


In [4]:
test = df.loc[df['dataset'] =='test']
test

,transaction_descriptor,store_number,dataset
200,IN-N-OUT BURGER #242,242,test
201,BP#9442088LIBERTYVILLE B,9442088,test
202,JCPENNEY 1419,1419,test
203,ROSS STORES #1019,1019,test
204,WM SUPERCENTER #38,38,test
...,...,...,...
295,MCDONALD'S F2151,F2151,test
296,NST BEST BUY #1403 332411,1403,test
297,CVS/PHARMACY #06689,6689,test
298,BANANA REPUBLIC #8109,8109,test


In [5]:
teks = train.transaction_descriptor.tolist()
number = train.store_number.tolist()

In [6]:
import re

In [7]:
def find_matches(text,number):
    match = re.search(str(number),text)
    start = match.start()
    end = match.end()
    return(start, end)

find_matches(str(teks), str(number))

(1, 2)

In [8]:
train_data = []

for item in train.values:
    start, end = find_matches(item[0],item[1])
    train_item = (item[0],{
          'entities':[(start, end, 'NUMBER')]
    })

    train_data.append(train_item)

train_data

[('DOLRTREE 2257 00022574 ROSWELL', {'entities': [(9, 13, 'NUMBER')]}),
 ('AUTOZONE #3547', {'entities': [(10, 14, 'NUMBER')]}),
 ('TGI FRIDAYS 1485 0000', {'entities': [(12, 16, 'NUMBER')]}),
 ('BUFFALO WILD WINGS 003', {'entities': [(21, 22, 'NUMBER')]}),
 ('J. CREW #568 0', {'entities': [(9, 12, 'NUMBER')]}),
 ('KRISPY KREME 0040        GREENVILLE   SC',
  {'entities': [(15, 17, 'NUMBER')]}),
 ('FIVE GUYS MN 1847 ECOM   612-339-9733 MN',
  {'entities': [(13, 17, 'NUMBER')]}),
 ('CASEYS GEN STORE 2650', {'entities': [(17, 21, 'NUMBER')]}),
 ('HUDDLE HOUSE 535', {'entities': [(13, 16, 'NUMBER')]}),
 ('JAMBA JUICE #1305', {'entities': [(13, 17, 'NUMBER')]}),
 ('ANN TAYLOR FACTORY #2202', {'entities': [(20, 24, 'NUMBER')]}),
 ('Subway 26824', {'entities': [(7, 12, 'NUMBER')]}),
 ('MARSHALLS #0688', {'entities': [(12, 15, 'NUMBER')]}),
 ('OREILLY AUTO #4681', {'entities': [(14, 18, 'NUMBER')]}),
 ('TA #227 BARSTOW REST', {'entities': [(4, 7, 'NUMBER')]}),
 ('SONIC #3207', {'entities': [(

In [9]:
import spacy
from tqdm import tqdm

In [10]:
nlp = spacy.blank('en') 
ner = nlp.create_pipe('ner') 
nlp.add_pipe(ner)

In [11]:
import random

In [12]:
for _, annotations in train_data: 
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes): 
    optimizer = nlp.begin_training()
    for itn in range(30):
        random.shuffle(train_data)
        losses = {}
        for text, annotations in tqdm(train_data):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

100%|██████████| 200/200 [00:05<00:00, 36.12it/s]


{'ner': 139.5420432683508}


100%|██████████| 200/200 [00:05<00:00, 36.63it/s]


{'ner': 25.69481070868198}


100%|██████████| 200/200 [00:05<00:00, 35.33it/s]


...

{'ner': 2.76368576436498}


100%|██████████| 200/200 [00:09<00:00, 20.75it/s]


{'ner': 0.00013547585192785503}


100%|██████████| 200/200 [00:09<00:00, 20.58it/s]

{'ner': 1.5039316305863469e-05}


In [13]:
for text, _ in train_data:
    doc = nlp(text)
    ent = [(ent.text) for ent in doc.ents]
    print(ent)

['1970']
['EXPRESS#0098']
['13822']
['STORE472605']
[]
['729724']
['14782798']
['FAMOUSFOOTWEAR#570903']
['723832']
['483280353']
['233']
['5754']
['G020029']
['1085']
['0780350500']
['3857']
['440']
['337734']
['7591']
['1305']
['3162']
['295']
[]
['500']
['25']
[]
['284']
['304']
[]
[]
['4215']
['2989']
['PETRO045518']
['974']
['14639']
[]
['10262660']
['0142']
['380']
['3207']
['1040']
['01']
['1847']
['9502']
['1338']
['0354']
['072473']
[]
['F6297']
['F26490']
[]
['029']
['0662']
['46221']
['3547']
['15474']
['409']
['2650']
['F565']
[]
['687']
['1563']
['13360692']
['1783']
['479']
[]
['029071']
[]
['0113']
['26824']
['0447']
['421']
['29']
[]
['7430']
[]
['22357']
['F16829']
['11789']
['2']
['7350']
['9386']
['209']
['6425']
['4204']
['25139']
['1327']
['730']
['3626']
['F1013']
['12862950']
[]
['833']
['15']
['1129']
['2050']
['0303']
['196']
['F3172']
[]
['0040']
['05320']
['07134']
['02282']
['4611']
['568']
['6470']
[]
[]
['9505']
['0976']
[]
['8550']
['231594']
['0748361300

In [14]:
test = df.loc[df['dataset']=='test']

test_data = []

for item in test.values:
    start, end = find_matches(item[0],item[1])
    test_item = (item[0],{
          'entities':[(start, end, 'NUMBER')]
    })

    test_data.append(test_item)

result = []
for text, _ in test_data:
    doc = nlp(text)
    ent = [(ent.text) for ent in doc.ents]
    result.append(ent)
    print(ent)
    

['242']
[]
['1419']
['1019']
['38']
[]
['629']
['1475']
['2505']
['825']
['2923']
['058']
['2039']
['382']
['012260']
['864']
[]
['011']
['70360265']
['4393']
['62']
['7212']
['30']
['WRENTHA130571']
['14178']
['3220']
['407']
['14']
['2454']
['17871401']
['3192']
['862751']
['1']
[]
['3889']
['401']
['2597']
['1253']
['507']
['472831773']
['277']
['59']
['48']
['231']
['11332']
['16482149']
['F33735']
['F671']
['F11370']
['50']
['11820']
['6619']
['6102']
['F2383']
['733780']
['62132001']
['F33124']
['15392']
['860977']
['F122']
['0982']
['188']
['15843']
['2742643']
['FOOTWEAR001261']
['EXPRESS#0813']
['7414']
[]
['392']
['66165']
['0647']
['14788']
['57331']
['34493']
['408']
['10101']
[]
['405']
['601128']
['4633']
['1997']
['21000236']
['FOOTWEAR730376']
['PETRO170928']
['0104235700']
['FAMOUSFOOTWEAR#132427']
['11966']
[]
['401']
['51']
['UT044']
[]
[]
[]
['513']
['F2151']
['1403']
['06689']
['8109']
['0443']


In [15]:
test['result'] = result
test.head(15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,transaction_descriptor,store_number,dataset,result
200,IN-N-OUT BURGER #242,242,test,[242]
201,BP#9442088LIBERTYVILLE B,9442088,test,[]
202,JCPENNEY 1419,1419,test,[1419]
203,ROSS STORES #1019,1019,test,[1019]
204,WM SUPERCENTER #38,38,test,[38]
205,TUESDAY MORNING # 0673 06,673,test,[]
206,IHOP 629 WHITE HOUSE TN,629,test,[629]
207,LBOUTLETS#4249 1475 N BUR,4249,test,[1475]
208,"WINN DIXIE #2505 VALRICO, FL (3454)",2505,test,[2505]
209,BURLINGTON STORES 825,825,test,[825]


In [16]:
test.to_csv('result.csv')